In [ ]:
from scidash_api import rest_client, settings

In [ ]:
base_url = settings.get_config('ip')

rest_client = rest_client.ScidashRestApiClient(base_url=base_url, username='admin', password='admin_test')

In [ ]:
test_import_path = 'neuronunit.tests.passive.InputResistanceTest'
test_instance = {
    'name': 'Name of the Test instance',
    'description': 'Description of the Test instance',
    'tags': [],
    'observation' : {
        'n':'1',
        'std':'50',
        'mean':'100'},
    'params' : {
        'dt':0.000025,
        'tmax':'10',
        'delay':0.1,
        'padding':0.2,
        'duration':0.3,
        'amplitude':-1.0000000000000001e-11},
}

# create a test instance and store it for later
new_test_instance = rest_client.create_test(
    test_instance=test_instance,
    test_class_import_path=test_import_path)

In [ ]:
# create a model instance
model_class_url = 'https://raw.githubusercontent.com/scidash/neuronunit/metacell/neuronunit/models/NeuroML2/LEMS_2007One.xml'
# get all model classes from SciDash
model_classes = rest_client.get_model_classes(model_class_url)
model_class_name = 'ReducedModel'
# get the 'reducedmodel' model class from the model classes list
model_class = list(filter(lambda mc: (mc['class_name'] == model_class_name), model_classes))[0]

model_instance = {
   "tags":[],
   "backend":None,
   "name":"Name of the Model instance",
   'url':model_class_url,
   "run_params":{
      "stateVariables":[
         "net1.RS_pop[0].u",
         "net1.RS_pop[0].iSyn",
         "net1.RS_pop[0].iMemb",
         "net1.RS_pop[0].v",
         "net1.explicitInput.RS_Iext.i",
         "time"
      ],
      "watchedVariables":[
         "net1.RS_pop[0].u",
         "net1.RS_pop[0].v"
      ],
      "params":[
         {
            "name":"Model.neuroml.RS.v0",
            "value":"-60",
            "unit":"mV"
         },
         {
            "name":"Model.neuroml.RS.k",
            "value":"0.699999",
            "unit":"nS_per_mV"
         },
         {
            "name":"Model.neuroml.RS.vr",
            "value":"-60",
            "unit":"mV"
         },
         {
            "name":"Model.neuroml.RS.vt",
            "value":"-40",
            "unit":"mV"
         },
         {
            "name":"Model.neuroml.RS.vpeak",
            "value":"35",
            "unit":"mV"
         },
         {
            "name":"Model.neuroml.RS.a",
            "value":"0.029999",
            "unit":"per_ms"
         },
         {
            "name":"Model.neuroml.RS.b",
            "value":"-2",
            "unit":"nS"
         },
         {
            "name":"Model.neuroml.RS.c",
            "value":"-50",
            "unit":"mV"
         },
         {
            "name":"Model.neuroml.RS.d",
            "value":"100",
            "unit":"pA"
         },
         {
            "name":"Model.neuroml.RS.C",
            "value":"100",
            "unit":"pF"
         },
         {
            "name":"Model.neuroml.RS_Iext.delay",
            "value":"0",
            "unit":"ms"
         },
         {
            "name":"Model.neuroml.RS_Iext.duration",
            "value":"520",
            "unit":"ms"
         },
         {
            "name":"Model.neuroml.RS_Iext.amplitude",
            "value":"100",
            "unit":"pA"
         }
      ]
   }
}

# create the model and store it for later
new_model_instance = rest_client.create_model(model_instance=model_instance, model_class=model_class)


In [ ]:
# schedule the test and model
# for suites you can pass multiple test and model instances and the name of the new suite
rest_client.schedule(
    suite_name='',
    test_instances=[new_test_instance,],
    model_instances=[new_model_instance,])
